Photometry
Version 0.1 Sep 2019

Use this notebook in conjunction with Activity 5.5 Open photometry results in Python and Activity 6.4 Generate CMDs

You may also wish to refer back to the Hyades_CMD notebook from Week 3.

This notebook is in two parts.

In the first section you will import results from an AstroImageJ measurements file and inspect the data.

In the second section you will use Eq 3.1b to calculate magnitudes and then plot the CMD.

1 Data inspection
The photometry in AstroimageJ produces a file containing the relative fluxes of the target stars, together with some other information. The default name for this file is Measurements.xls. In spite of the .xls extension this is actually a text file, similar to a .csv file but with fields separated by tabs rather than commas. After completing Activity 5.4 you will have a measurements file containing the results from your aperture photometry of the two images of M13 from the sample dataset.

We can read this file using Pandas in the same way as a .csv file, but specifying \t as the separator to denote a tab-delimited file.

The AstroImageJ measurements file contains some columns of data (rel_flux_err) that are not needed. Pandas allows us to specify which columns to import, using a helper function that returns True to include a column, and False to exclude a column. Refer back to the UsingPandas notebook from Skills week 1 as necessary.

This notebook also uses regular expressions to identify the rows in the data file that contain values from the B and V filters. Refer to the UsingRegex notebook from the Skills weeks for information on using Regular expressions.

The following cell contains an example program that carries out the necessary steps to import the data from your measurements file. Modify this as necessary (for instance, if your file has a different name) and work through the steps making sure that you understand how it works. Use the commented-out print(...) statements to inspect the format of the data file at each step.


In [2]:

# First import the packages that we will need
import pandas as pd
import re     # for Regular expressions

# Define helper function for reading in required columns
# pd.read_csv calls this once for each column - returns True
# if we want to include this column, and False if not wanted
# (this allows us to include the relative flux columns
#  and to exclude the rel-flux_err columns that contain error values)
def isValidCol(colName):
    if (colName[0:10] == 'rel_flux_T' or colName == 'Label'):
        return True
    else:
        return False

# File name is defined here
# Change this for a different filename
# For Activity 6.4 you may wish to include a question 
# to prompt the user for a filename
strFileName = "Measurements M13 in b-v filter.xls" 

# Read data from the file into a Pandas DataFrame
ClusterData = pd.read_csv(strFileName, sep = '\t', usecols = isValidCol)
#print(ClusterData)

# The first row of the dataset contains column headings
# Subsequent rows contain the photometry results from each image
# Normally there will be two rows - one for the B filter and one for the V filter
# The next step is to identify which row is which.  This can be done using regular expressions.

# First, define a search string to isolate 'B' or 'V' in the correct place
# The image name will contain either "Filter_B" or "Filter_V"
# r at the start of the string defines a raw string (need for regular expression)
strFilter = r'.*Filter_([BV])_.*'

# Assuming this will ALWAYS match, replace image name with either 'B' or 'V'
ClusterData['Label'] = [re.search(strFilter, x).group(1) for x in ClusterData['Label']]

# The label column now contains just 'B' or 'V' - we can use this as an index
ClusterData.set_index('Label', inplace = True)
#print(ClusterData)

# Finally, transpose from rows to columns - this makes it easier to work with the dataset
ClusterData = ClusterData.transpose()

print(ClusterData)


Label                V         B
rel_flux_T1   0.018866  0.026757
rel_flux_T2   0.013772  0.025055
rel_flux_T3   0.017937  0.047944
rel_flux_T4   0.027508  0.061131
rel_flux_T5   0.016118  0.022152
rel_flux_T6   0.024936  0.036405
rel_flux_T7   0.024540  0.035607
rel_flux_T8   0.029992  0.053711
rel_flux_T9   0.024647  0.035122
rel_flux_T10  0.018374  0.044943
rel_flux_T11  0.046438  0.076898
rel_flux_T12  0.064427  0.087755
rel_flux_T13  0.070675  0.089748
rel_flux_T14  0.014510  0.020428
rel_flux_T15  0.013933  0.041806
rel_flux_T16  0.036190  0.058874
rel_flux_T17  0.019607  0.026596
rel_flux_T18  0.028918  0.043891
rel_flux_T19  0.021855  0.029777
rel_flux_T20  0.025649  0.036938
rel_flux_T21  0.041533  0.058203
rel_flux_T22  0.023015  0.032566
rel_flux_T23  0.017219  0.025836
rel_flux_T24  0.035216  0.049430
rel_flux_T25  0.031617  0.062726
rel_flux_T26  0.035113  0.050004
rel_flux_T27  0.150339  0.254134
rel_flux_T28  0.598774  0.839873
rel_flux_T29  0.046176  0.077004
rel_flux_T


You now have the photometry data imported. You can carry out calculations on the data and add new columns for the results. Remember that in Pandas, calculations can operate on an entire column of data at once.

For instance: ClusterData['FluxRatio'] = ClusterData['B']/ClusterData['V'] would create a new column labelled 'FluxRatio'

Try this now, by running the next cell:


In [ ]:

# This step is not needed for the calculations in Activity 6.4
# It is simply an illustration of how to carry out calculations and 
# add columns to the dataset
ClusterData['FluxRatio'] = ClusterData['B']/ClusterData['V']

print(ClusterData)



This completes the work for Activity 5.5. You now have a notebook that will import results from an AstroImageJ measurements file, ready to use with the photometry data from your own cluster images.

Save this notebook and return to it in Activity 6.4.

2 Calculate magnitudes and plot CMD
The values in the AstroImageJ results file are relative fluxes - in other words, each value is the brightness of the target star (in the B or V band) divided by the brightness of the reference. To produce the Colour Magnitude Diagram, these need to be converted to apparent magnitudes. You can do this using Eq 3.1b together with the catalogue magnitude values for your chosen reference star, as described in Activity 6.4.

You can use np.log10(...) to calculate logs to base 10.

The next cell contains an outline program segment that follows on from importing the data using the first cell in Section 1. This routine carries out the magnitude calculations on the imported data and plots the CMD. Complete this cell by filling in the lines where indicated. Refer back to the Hyades_CMD notebook from Activity 3.2 and to the Matplotlib exercises from the Skills weeks as needed for advice on graph plotting.


In [ ]:

# Complete the program by adding lines where indicated by "<<<<"
# This cell should be run _after_ importing the dataset using 
# your program from Activity 5.5 in the first cell.

# First, import any packages not already imported
import matplotlib.pyplot as plt
import numpy as np

# From the first cell you already have the photometry data in the form of 
# two columns containing the B and V relative fluxes.
# For the CMD you need to create new columns containing the V magnitude 
# and the colour index (B-V) for each of the target stars.

# TODO: Complete the following lines with the catalogue B and V 
# magnitudes of your reference star
REF_B_MAG =                              # <<<<
REF_V_MAG =                              # <<<<


# TODO: complete the following lines using Eq 3.1b and the reference magnitudes

ClusterData['Bmag'] =                    # <<<<
ClusterData['Vmag'] =                    # <<<<
ClusterData['B_V']  =                    # <<<<
# print(ClusterData)

# TODO: Finally, create a plot of the V magnitude against the colour index (B-V)
# Remember to label and annotate your plot appropriately.
# You will also need to set axis limits to get the magnitude scale 
#    running in the correct direction and possibly to exclude outliers.
# You may wish to export the plot as a .png for inclusion in your TMA
# <<<<
# <<<<
# <<<<



Once you have completed your program and checked the calculations, repeat the processing for each of the clusters that you have imaged.

You can either keep the notebook in its current form, running each of the cells in turn, or combine the steps to give a complete program in one cell.

If you wish, you can enhance your program by using techniques from earlier notebooks to allow the user to select which cluster to plot.